In [ ]:
import pandas as pd
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

# check if GLPK is installed. If not, install.
if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk
        except:
            pass
assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

# Formulación de mezclas óptima
Dado un conjunto de productos químicos y un conjunto de restricciones de propiedades, determinar la mezcla y/o combinación óptima de los diferentes compuestos que resulta en el menor coste posible.

## Datos

In [ ]:
d = {'Coste':{'A': 2. , 'B': 2.  , 'C': 5.  }, 
     'VitA': {'A': .5 ,'B': .4, 'C': .3 }, 
     'VitB': {'A': .2 , 'B': .1 , 'C': .3 }
     }
df = pd.DataFrame.from_dict(d, orient='index')

display(df)
print('Cantidad máxima de vitamina A:', 0.4)
print('Cantidad mínima de vitamina B:', 0.2)
print('Componenetes A y B son incompatibles')

In [ ]:
data = {None: {'C':  , 
               'P': , 
               'VitA':  ,
               'VitB':  ,
               'VitA_UB': ,
               'VitB_LB': , 
               }} 

## Implementación del modelo
El primer paso consiste en importar Pyomo y definir el tipo de modelo que se va a utilizar. Para este problema vamos a utilizar un modelo tipo `ConcreteModel`.

In [ ]:
import pyomo.environ as pyo

model  = pyo.ConcreteModel()

### Sets
En este ejemplo vamos a aprender a definir subconjuntos o subsets. Pyomo permite hacer esto de forma programática, utiilizando una función o filtro. 

A continuación, define un set que contenga una lista con los componentes que hay disponibles para mezclar y un subset en el que cada elemento sea una pareja de compuestos incompatibles (usa una estructura de datos de tupla `(a,b)`).

In [ ]:
model.C = pyo.Set ( initialize = , # TO DO
                    doc = 'Componentes') 

def incompatible_filter(model, i, j):
    return 
model.C_incompat = pyo.Set( initialize= ,# TO DO 
                            filter= ,# TO DO  
                            doc = 'Pares incompatibles')

In [ ]:
model.C_incompat.pprint()
model.C.pprint()

### Variables

In [ ]:
model.x = 
model.y = 

### Parámetros

In [ ]:
model.P = 
model.VitA = 
model.VitB = 
model.VitA_UB = 
model.VitB_LB = 
model.BigM = 

### Funcion objetivo
$$
\begin{align}
\text{Coste total} & = \sum_{c\in C} x_{c} P_{c} \nonumber
\end{align}
$$

In [ ]:
model.obj = 

### Restricciones estructurales
- Balance de materia
$
\begin{align}
\sum_{c\in C} x_{c} & = 1 \nonumber
\end{align}
$

In [ ]:
model.massfraction = 

- Especificaciones del producto final
$
\begin{align}
\sum_{c\in C} x_{c}VitA_{c} & \leq VitA^{UB} \nonumber \\
\sum_{c\in C} x_{c}VitB_{c} & \geq VitB^{LB} \nonumber
\end{align}
$

In [ ]:
model.comp_ub = 
model.comp_lb =  

# Restricciones lógicas: incompatibilidad de componentes

- Reformulación Big-M
$
\begin{align}
x_A & \leq M\;y \nonumber \\
x_B & \leq (1-M)\;y \nonumber
\end{align}
$

Además del comando `Constraint`, diferentes restricciones se pueden agrupar utilizando una `ConstraintList`.

In [ ]:
model.ref_BigM = pyo.ConstraintList()
for pair in model.C_incompat:
    a, b = pair
    model.ref_BigM.add( ) # TO DO 
    model.ref_BigM.add( ) # TO DO    

- Disyunciones

Antes de utilizar la enxtensión para GDP de Pyomo, hay que importar las funciones correspondientes. Para modelar la siguiente disyunción, se necesitarán los comandos `Disjunct`, `Disjunction` y, más adelante, `TransformationFactory`.

$$
\left[
\begin{array}{c}
Y \\
x_B=0
\end{array}
\right] 
\lor 
\left[
\begin{array}{c}
\neg Y \\
x_A=0
\end{array}
\right]
$$

In [ ]:
# Importar funciones necesarias
from pyomo.gdp import Disjunct, Disjunction

# Término que se cumple si Y = True
model.term1 = Disjunct()
model.term1.consA = pyo.Constraint()

# Término que se cumple si Y = False
model.term2 = 
model.term2.consB = 

# Añadir restricción en forma de disyunción
model.disyuncion = Disjunction(expr = ) # TO DO

## Resolución
El solver que se va a utilizar en la optimización de este problema es Cbc (Coin-or branch and cut). La nomenclatura para llamarlo es `SolverFactory('cbc')`.

### Reformulación Big M

In [ ]:
# Desactivar restricciones
model.term1.deactivate()
model.term2.deactivate()
model.disyuncion.deactivate()

# Llamada al solver
opt = pyo.SolverFactory('glpk')
results = opt.solve(model)

In [ ]:
for c in model.C:
    print(f"{c} = {model.x[c]()}")

### Extensión Pyomo GDP
Aplica el método de la envolvente convexa para reformular la disyunción que aparece en este problema. La sintaxis que debes utilizar es `TransoformationFactory(trf_name).apply_to(model_name)`.

In [ ]:
# # Acivar/desactivar restricciones
# model.term1.activate()
# model.term2.activate()
model.disyuncion.activate()

model.ref_BigM.deactivate()

# Aplicar tranformación
transformation = 'hull'
if transformation == 'bigm':
    pyo.TransformationFactory('gdp.bigm').apply_to(model)
elif transformation == 'hull':
    pyo.TransformationFactory('gdp.hull').apply_to(model)
  
# Llamada al solver
opt = pyo.SolverFactory('glpk')
results = opt.solve(model)

In [ ]:
for c in model.C:
    print(f"{c} = {model.x[c]()}")